# A - Azure ML Studio: Workspace erstellen

Environment für dieses Notebook: `ads-ml-light` (*requirements-py3.11-ads-ml-light.txt*).

In diesem Modul nutzen wir die [Azure Machine Learning Python SDK (v2)](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace?view=azureml-api-2&tabs=python), um Ressourcen auf Azure programmatisch zu erstellen.

## 1. Einmalig: Anmeldung bei Azure + Freischaltung "Azure for Students"

Meldet euch mit euren DBU Credentials bei [portal.azure.com](https://portal.azure.com). Aus dem Portal erfahrt ihr eure `subscription_id`, diese muss in die Datei `.env` kopiert werden.

## 2. Umgebungsvariablen

Die `subscription_id` ist für jeden Student einzigartig - solche Werte behandeln wir daher wie "Geheimnisse" und legen sie nicht lesbar im Notebook ab.

Mit dem `dotenv` Paket lesen wir die Umgebungsvariablen aus der Datei `.env ` ein. Das Paket sucht nach dieser File in den geöffneten Verzeichnissen, üblicherweise liegt die File jedoch gut sichtbar auf der ersten Ebene. Erstellt euch diese File oder nutzt das Template aus den gezippten Downloads.

### `.env` Inhalt

```bash
AZURE_SUBSCRIPTION_ID="<YOUR-SUBSCRIPTION-ID>"
```

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")

In [3]:
# kurzer Check, ob es geklappt hat
assert subscription_id is not None

## 3. Anmelden bei Azure

In der Environment ist auch die azure-cli installiert. Mit aktivierter Environment in der Command Line könnt ihr euch einloggen via `az login`, oder ihr führt die nächste Zelle aus.

<span style="color:orange">**Ohne erfolgreichen Login schlagen die folgenen Notebook Zellen fehl!**</span>

In [ ]:
!az login

## 4. ML Client zur Interaktion mit Azure und der Python SDK

In [4]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group_name="rg-adsc21")

## 5. Resource Group erstellen

Falls schon eine Resource Group vorhanden ist, kann diese genutzt werden.

[Azure Doku](https://learn.microsoft.com/en-us/azure/developer/python/sdk/examples/azure-sdk-example-resource-group)

In [5]:
from azure.mgmt.resource import ResourceManagementClient

resource_client = ResourceManagementClient(DefaultAzureCredential(), subscription_id)

# Provision the resource group.
rg_create = resource_client.resource_groups.create_or_update(
    "rg-adsc21", {"location": "westeurope"}
)

print(f"Provisioned resource group {rg_create.name} in the {rg_create.location} region")

Provisioned resource group rg-adsc21 in the westeurope region


## 6. Machine Learning Workspace erstellen

In [5]:
from azure.ai.ml.entities import Workspace

ws = Workspace(
    name="workspace-ads-ml2",
    resource_group="rg-adsc21",
    display_name="ADS ML Workspace",
    description="Machine Learning Workspace erstellt mit der Azure ML Client SDK.",
)

In [6]:
ws = ml_client.workspaces.begin_create(ws)

The deployment request workspace-ads-ml2-1978382 was accepted. ARM deployment URI for reference: 
https://portal.azure.com//#blade/HubsExtension/DeploymentDetailsBlade/overview/id/%2Fsubscriptions%2F083a9eb0-a7cf-465e-9bdd-e3481c3d4ad9%2FresourceGroups%2Frg-adsc21%2Fproviders%2FMicrosoft.Resources%2Fdeployments%2Fworkspace-ads-ml2-1978382


In [7]:
ws = ws.result()
print(ws)

Creating Storage Account: (workspacstorage887ed16da  ) ...  Done (22s)
Creating Key Vault: (workspackeyvault66a954b6  )  Done (18s)
Creating Log Analytics Workspace: (workspaclogalyti9bab6dae  )  Done (16s)
Creating AzureML Workspace: (workspace-ads-ml2  ) ..  Done (19s)
Creating Application Insights: (workspacinsights7fa88c88  )  Done (19s)
Total time : 43s



application_insights: /subscriptions/083a9eb0-a7cf-465e-9bdd-e3481c3d4ad9/resourceGroups/rg-adsc21/providers/Microsoft.insights/components/workspacinsights7fa88c88
description: Machine Learning Workspace erstellt mit der Azure ML Client SDK.
discovery_url: https://westeurope.api.azureml.ms/discovery
display_name: ADS ML Workspace
enable_data_isolation: false
hbi_workspace: false
id: /subscriptions/083a9eb0-a7cf-465e-9bdd-e3481c3d4ad9/resourceGroups/rg-adsc21/providers/Microsoft.MachineLearningServices/workspaces/workspace-ads-ml2
identity:
  principal_id: a3cab380-6130-4db8-9b6c-c35361dc8706
  tenant_id: b68f9931-79d6-4e5b-bf56-72ad3eb203ca
  type: system_assigned
key_vault: /subscriptions/083a9eb0-a7cf-465e-9bdd-e3481c3d4ad9/resourceGroups/rg-adsc21/providers/Microsoft.Keyvault/vaults/workspackeyvault66a954b6
location: westeurope
managed_network:
  isolation_mode: disabled
  outbound_rules: []
mlflow_tracking_uri: azureml://westeurope.api.azureml.ms/mlflow/v1.0/subscriptions/083a9eb0-